In [1]:
'''
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/tcc')
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nos.chdir('/content/drive/My Drive/Colab Notebooks/tcc')\n"

In [2]:
import tensorflow as tf
import pickle
import pandas as pd

from modules.preprocessing import *

In [3]:
class EmotionDetect:
    def __init__(self, vectors_name, w_folder):
        self.w_folder = w_folder
        self.size_word2vec = 300
        self.max_sequences = 35
        # self.vectors = pickle.load(open('w2v_stop'+str(self.size_word2vec)+'.pkl', 'rb'))
        self.vectors = pickle.load(open(vectors_name, 'rb'))
        self.emotions = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
        self.tokenizer = Tokenizer()
        self.loadModels()

    def model_gru(self, embedding_matrix, embed_size):
        # load json and create model
        json_file = open('models/bi-gru300.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        return tf.keras.models.model_from_json(loaded_model_json)

    def loadModels(self):
        self.models = dict()
        for s in self.emotions:
            self.models[s] = self.model_gru(self.vectors.wv.vectors, self.size_word2vec)
            self.models[s].load_weights('models/'+s+ '_weights.h5')
    
    def getIndex(self, t):
        try:
            return self.vectors.wv.vocab[t].index
        except:
            return 0

    def predict(self, text):
        x = self.tokenizer.cleanText(text)
        x = [self.getIndex(t) for t in x]
        x = tf.keras.preprocessing.sequence.pad_sequences([x], maxlen=self.max_sequences)
        return dict(zip(self.emotions, [self.models[s].predict(x)[0][0] for s in self.emotions]))

In [4]:
classifier = EmotionDetect('models/ft_300.pkl', 'models')

In [5]:
from datetime import timedelta, date
!pip install GetOldTweets3
import GetOldTweets3 as got

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

def get_data(word, year=2020, month=1):

    start_date = date(year, month, 1)
    end_date = date(year, month+1, 1)

    tweets = []
    for single_date in tqdm(daterange(start_date, end_date), total=(end_date - start_date).days):
        # print(single_date)
        day = single_date.strftime("%Y-%m-%d")
        dayPlus = (single_date + timedelta(days=1)).strftime("%Y-%m-%d")

        tweetCriteria = got.manager.TweetCriteria().setQuerySearch(word)\
                                            .setSince(day)\
                                            .setUntil(dayPlus)\
                                            .setLang('en')\
                                            .setTopTweets(True)\
                                            .setMaxTweets(1000)
        tweet = got.manager.TweetManager.getTweets(tweetCriteria)

        for t in (tweet):
            emo = classifier.predict(t.text)
            tweets.append({
                        'date': t.date,
                        'id': t.id,
                        'text': t.text,
                        'favorites': t.favorites,
                        'retweets': t.retweets,
                        'replies': t.replies,

                        'anger' : int(emo['anger'] > 0.22),
                        'disgust' : int(emo['disgust'] > 0.1),
                        'fear': int(emo['fear'] > 0.1),
                        'joy' : int(emo['joy'] > 0.1),
                        'sadness' : int(emo['sadness'] > 0.1),
                        'surprise': int(emo['surprise'] > 0.13),

                        'anger_pred': emo['anger'],
                        'disgust_pred' : emo['disgust'],
                        'fear_pred': emo['fear'],
                        'joy_pred': emo['joy'],
                        'sadness_pred': emo['sadness'],
                        'surprise_pred': emo['surprise']
                        })
        aux = pd.DataFrame(tweets)
        aux.to_csv('data/data_autosave.csv')
    return tweets


In [6]:
def get_plot(search, start_m=1, start_y=2019, end_m=1, end_y=2020):
    for y, m in month_year_iter(start_m, start_y, end_m, end_y):
        name = str(y)+str(m)
        tweets = get_data(search, year=y, month=m)
        df = pd.DataFrame(tweets)
        df.to_csv('data/tweets_'+search+'_'+name+'.csv')
        df['date'] = pd.to_datetime(df['date']).dt.normalize()
        result = df.set_index('date')[['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']].resample("D").sum()

        plot = result.plot.area(stacked=True, figsize=(20,10))
        plot.figure.savefig('figures/sent_'+search+'_'+name+'.jpg')

In [7]:
# get_plot('guns', 8, 2019, 9, 2019) #relate to trump

In [ ]:
get_plot('guns', 3, 2020, 4, 2020) #relate to trump ?

 32%|███▏      | 10/31 [27:24<1:00:05, 171.70s/it]

In [ ]:
get_plot('virus', 1, 2020, 4, 2020)

In [ ]:
get_plot('home', 1, 2020, 4, 2020)

In [ ]:
get_plot('school', 3, 2020, 5, 2020)

In [ ]:
get_plot('trump', 11, 2016, 12, 2016) #elections?

In [ ]:
get_plot('trump', 4, 2019, 4, 2020) #elections?